In [40]:
import requests
import pandas as pd
import numpy as np
import json
import time

In [41]:
apiKey = "IaF6zOUaj8VG4ambmUwOodEFaxBnrR3A"
apiUrl = "https://aeroapi.flightaware.com/aeroapi/"
auth_header = {'x-apikey':apiKey}

# GET flights from DMK airport

In [42]:
Airport_id = 'DMK'
Airport_start = '2022-12-01T00:00:00Z'
Airport_end = '2022-12-04T00:00:00Z'

# Y-M-D
# 1) 2022-12-01 - 2022-12-04


In [43]:
#----- get flight data from DMK airport -----#

response = requests.get(apiUrl + f"airports/{Airport_id}/flights?start=2022-12-06T00%3A00%3A00Z&end=2022-12-07T00%3A00%3A00Z",
 headers=auth_header)

if response.status_code == 200:
    print(response.json())
else:
    print("Error executing request")

{'arrivals': [{'ident': 'TLM219', 'ident_icao': 'TLM219', 'ident_iata': 'SL219', 'fa_flight_id': 'TLM219-1670180160-schedule-0880', 'operator': 'TLM', 'operator_icao': 'TLM', 'operator_iata': 'SL', 'flight_number': '219', 'registration': 'HS-LUY', 'atc_ident': None, 'inbound_fa_flight_id': 'TLM218-1670158253-airline-0025', 'codeshares': [], 'codeshares_iata': [], 'blocked': False, 'diverted': False, 'cancelled': False, 'position_only': False, 'origin': {'code': 'VABB', 'code_icao': 'VABB', 'code_iata': 'BOM', 'code_lid': None, 'timezone': 'Asia/Kolkata', 'name': "Chatrapati Shivaji Int'l", 'city': 'Mumbai', 'airport_info_url': '/airports/VABB'}, 'destination': {'code': 'VTBD', 'code_icao': 'VTBD', 'code_iata': 'DMK', 'code_lid': None, 'timezone': 'Asia/Bangkok', 'name': "Don Muang Int'l (Old Bangkok Int'l)", 'city': 'Bangkok', 'airport_info_url': '/airports/VTBD'}, 'departure_delay': -1740, 'arrival_delay': -480, 'filed_ete': 13500, 'scheduled_out': '2022-12-06T18:40:00Z', 'estimated_o

In [44]:
print(json.dumps(response.json(), indent=4))

{
    "arrivals": [
        {
            "ident": "TLM219",
            "ident_icao": "TLM219",
            "ident_iata": "SL219",
            "fa_flight_id": "TLM219-1670180160-schedule-0880",
            "operator": "TLM",
            "operator_icao": "TLM",
            "operator_iata": "SL",
            "flight_number": "219",
            "registration": "HS-LUY",
            "atc_ident": null,
            "inbound_fa_flight_id": "TLM218-1670158253-airline-0025",
            "codeshares": [],
            "codeshares_iata": [],
            "blocked": false,
            "diverted": false,
            "cancelled": false,
            "position_only": false,
            "origin": {
                "code": "VABB",
                "code_icao": "VABB",
                "code_iata": "BOM",
                "code_lid": null,
                "timezone": "Asia/Kolkata",
                "name": "Chatrapati Shivaji Int'l",
                "city": "Mumbai",
                "airport_info_url": "/air

In [45]:
Arrivals_FlightID = []
Departures_FlightID = []

In [46]:
#----- add flight data from DMK airport -----#

for i in response.json()['arrivals']:
    Arrivals_FlightID.append(i['fa_flight_id'])
    
for i in response.json()['departures']:
    Departures_FlightID.append(i['fa_flight_id'])

In [47]:
for item in Departures_FlightID:
    print(item)

NOK600-1670198400-schedule-0700
NOK340-1670198400-schedule-0684
SNY20-1670370011-adhoc-0
NOK122-1670197440-schedule-0484
AIQ3188-1670197440-schedule-0483
AIQ311-1670197440-schedule-0491
AIQ3029-1670197440-schedule-0464
AIQ3102-1670197440-schedule-0478
TLM702-1670196540-schedule-0499
AIQ396-1670195580-schedule-0471
NOK570-1670195580-schedule-0484
NOK100-1670195580-schedule-0495
NOK550-1670195580-schedule-0452
TLM600-1670195580-schedule-0482
TLM506-1670195580-schedule-0485


# GET data from flight ID

In [48]:
#----- get data from flight ID -----#
Dict_flights = {}

i = 1
for id in Departures_FlightID:
    response = requests.get(apiUrl + f"flights/{id}/track",
    headers=auth_header)

    if response.status_code == 200:
        print(i)
        #----- Add data to dict -----#
        Dict_flights[id] = pd.DataFrame.from_dict(response.json()['positions'])
        
        #----- check time -----#
        Dict_flights[id] = Dict_flights[id].drop_duplicates(subset=['timestamp'], keep='first')
        
        #----- interpolate 1 sec -----#
        Dict_flights[id]['timestamp'] = pd.to_datetime(Dict_flights[id].timestamp)
        nidx = np.arange(Dict_flights[id]['timestamp'][0], Dict_flights[id]['timestamp'].iloc[-1], 1000000 )
        nidx = pd.to_datetime(nidx)

        Dict_flights[id]['timestamp'] = Dict_flights[id]['timestamp'].round('S').dt.tz_localize(None)
        Dict_flights[id].set_index('timestamp', inplace=True)
        Dict_flights[id] = Dict_flights[id].reindex(Dict_flights[id].index.union(nidx))

        #----- interpolate ['altitude', 'groundspeed', 'heading', 'latitude', 'longitude'] -----#
        Dict_flights[id] = Dict_flights[id][['fa_flight_id', 'altitude', 'groundspeed', 'heading', 'latitude', 'longitude']]
        Dict_flights[id]['altitude'] = pd.to_numeric(Dict_flights[id]['altitude'])
        Dict_flights[id]['groundspeed'] = pd.to_numeric(Dict_flights[id]['groundspeed'])
        Dict_flights[id]['heading'] = pd.to_numeric(Dict_flights[id]['heading'])
        Dict_flights[id] = Dict_flights[id].interpolate(method='time',limit_direction='both',limit=100)
        Dict_flights[id]['fa_flight_id'] = id
        Dict_flights[id].reset_index(inplace=True)
            
    else:
        print("Error executing request")
        
    i+=1
    time.sleep(7)

1


C:\Users\Nopnpr\AppData\Local\Temp\ipykernel_2548\3004202832.py:19: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  nidx = np.arange(Dict_flights[id]['timestamp'][0], Dict_flights[id]['timestamp'].iloc[-1], 1000000 )


2
3
4
5
6
7
8
9
Error executing request
Error executing request
Error executing request
13
14
15


In [49]:
# Dict_flights['SNY20-1670370011-adhoc-0'].loc[:, ~Dict_flights['SNY20-1670370011-adhoc-0'].index.duplicated()]

In [54]:
Dict_flights.keys()

dict_keys(['NOK600-1670198400-schedule-0700', 'NOK340-1670198400-schedule-0684', 'SNY20-1670370011-adhoc-0', 'NOK122-1670197440-schedule-0484', 'AIQ3188-1670197440-schedule-0483', 'AIQ311-1670197440-schedule-0491', 'AIQ3029-1670197440-schedule-0464', 'AIQ3102-1670197440-schedule-0478', 'TLM702-1670196540-schedule-0499', 'NOK550-1670195580-schedule-0452', 'TLM600-1670195580-schedule-0482', 'TLM506-1670195580-schedule-0485'])

In [51]:
print(json.dumps(response.json(), indent=4))

{
    "positions": [
        {
            "fa_flight_id": null,
            "altitude": 3,
            "altitude_change": "C",
            "groundspeed": 167,
            "heading": 209,
            "latitude": 13.90343,
            "longitude": 100.59958,
            "timestamp": "2022-12-06T23:06:58Z",
            "update_type": "A"
        },
        {
            "fa_flight_id": null,
            "altitude": 8,
            "altitude_change": "C",
            "groundspeed": 170,
            "heading": 210,
            "latitude": 13.89217,
            "longitude": 100.59314,
            "timestamp": "2022-12-06T23:07:14Z",
            "update_type": "A"
        },
        {
            "fa_flight_id": null,
            "altitude": 15,
            "altitude_change": "C",
            "groundspeed": 179,
            "heading": 208,
            "latitude": 13.88017,
            "longitude": 100.58656,
            "timestamp": "2022-12-06T23:07:30Z",
            "update_type": "A"
     

In [52]:
df = pd.read_csv(f'flights_data/{id}.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'flights_data/TLM506-1670195580-schedule-0485.csv'

# test ไม่ใช้จ้า

In [ ]:
# Dict_flights = {}

# Dict_flights['AIQ642-1669939380-schedule-0476'] = pd.read_csv(f'flights_data/AIQ642-1669939380-schedule-0476.csv')
# Dict_flights['AIQ642-1669939380-schedule-0476']['fa_flight_id'] = 1

# Dict_flights['NOK340-1669938420-schedule-0407'] = pd.read_csv(f'flights_data/NOK340-1669938420-schedule-0407.csv')
# Dict_flights['NOK340-1669938420-schedule-0407']['fa_flight_id'] = 2

# Dict_flights['AIQ642-1669939380-schedule-0476'].head()

,fa_flight_id,altitude,altitude_change,groundspeed,heading,latitude,longitude,timestamp,update_type
0,1,4,C,179,208,13.89944,100.59736,2022-12-03T23:57:52Z,A
1,1,10,C,184,208,13.88798,100.59111,2022-12-03T23:58:08Z,A
2,1,17,C,186,208,13.87573,100.58441,2022-12-03T23:58:24Z,A
3,1,23,C,190,233,13.86502,100.57491,2022-12-03T23:58:40Z,A
4,1,28,C,196,276,13.86214,100.56075,2022-12-03T23:58:56Z,A


In [ ]:
# Dict_flights['AIQ642-1669939380-schedule-0476']

# Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'] = pd.to_datetime(Dict_flights['AIQ642-1669939380-schedule-0476'].timestamp)

# nidx = np.arange(Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'][0], Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'].iloc[-1], 1000000 )
# nidx = pd.to_datetime(nidx)
# # nidx

# Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'] = Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'].round('S').dt.tz_localize(None)
# Dict_flights['AIQ642-1669939380-schedule-0476'].set_index('timestamp', inplace=True)
# Dict_flights['AIQ642-1669939380-schedule-0476'] = Dict_flights['AIQ642-1669939380-schedule-0476'].reindex(Dict_flights['AIQ642-1669939380-schedule-0476'].index.union(nidx))

# Dict_flights['AIQ642-1669939380-schedule-0476'] = Dict_flights['AIQ642-1669939380-schedule-0476'][['altitude', 'groundspeed', 'heading', 'latitude', 'longitude']]
# Dict_flights['AIQ642-1669939380-schedule-0476']['altitude'] = pd.to_numeric(Dict_flights['AIQ642-1669939380-schedule-0476']['altitude'])
# Dict_flights['AIQ642-1669939380-schedule-0476']['groundspeed'] = pd.to_numeric(Dict_flights['AIQ642-1669939380-schedule-0476']['groundspeed'])
# Dict_flights['AIQ642-1669939380-schedule-0476']['heading'] = pd.to_numeric(Dict_flights['AIQ642-1669939380-schedule-0476']['heading'])
# Dict_flights['AIQ642-1669939380-schedule-0476'] = Dict_flights['AIQ642-1669939380-schedule-0476'].interpolate(method='time',limit_direction='both',limit=50)

# Dict_flights['AIQ642-1669939380-schedule-0476']

C:\Users\Nopnpr\AppData\Local\Temp\ipykernel_20784\1800424518.py:5: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  nidx = np.arange(Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'][0], Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'].iloc[-1], 1000000 )


,altitude,groundspeed,heading,latitude,longitude
2022-12-03 23:57:52,4.000,179.0000,208.0,13.899440,100.597360
2022-12-03 23:57:53,4.375,179.3125,208.0,13.898724,100.596969
2022-12-03 23:57:54,4.750,179.6250,208.0,13.898008,100.596579
2022-12-03 23:57:55,5.125,179.9375,208.0,13.897291,100.596188
2022-12-03 23:57:56,5.500,180.2500,208.0,13.896575,100.595798
...,...,...,...,...,...
2022-12-04 01:20:56,11.000,147.0000,107.0,21.243370,105.729980
2022-12-04 01:20:57,11.250,146.0000,107.0,21.243182,105.730627
2022-12-04 01:20:58,11.500,145.0000,107.0,21.242995,105.731275
2022-12-04 01:20:59,11.750,144.0000,107.0,21.242807,105.731922


In [ ]:
Dict_flights.keys()

dict_keys(['NOK600-1670198400-schedule-0700', 'NOK340-1670198400-schedule-0684', 'SNY20-1670370011-adhoc-0'])

In [ ]:
# Dict_flights['SNY20-1670370011-adhoc-0'].reset_index(inplace=True)
# Dict_flights['SNY20-1670370011-adhoc-0']

,timestamp,fa_flight_id,altitude,altitude_change,groundspeed,heading,latitude,longitude,update_type
0,2022-12-06 23:49:06,None,3,C,81,209,13.90590,100.60499,A
1,2022-12-06 23:49:06,None,2,D,81,209,13.90590,100.60499,A
2,2022-12-06 23:49:22,None,4,C,91,208,13.90027,100.60200,A
3,2022-12-06 23:49:38,None,6,C,92,226,13.89427,100.59835,A
4,2022-12-06 23:49:54,None,7,C,92,274,13.89207,100.59193,A
...,...,...,...,...,...,...,...,...,...
173,2022-12-07 00:53:21,None,12,D,112,208,13.97526,100.64373,X
174,2022-12-07 00:54:25,None,7,D,94,207,13.94920,100.62908,A
175,2022-12-07 00:54:41,None,6,D,91,208,13.94263,100.62556,A
176,2022-12-07 00:54:57,None,5,D,91,209,13.93703,100.62255,A


In [ ]:
Dict_flights['SNY20-1670370011-adhoc-0'] = Dict_flights['SNY20-1670370011-adhoc-0'].drop_duplicates(subset=['timestamp'], keep='first')

In [63]:
Dict_flights['SNY20-1670370011-adhoc-0'].reset_index(inplace=True)

In [67]:
Dict_flights['SNY20-1670370011-adhoc-0'].rename(columns={'index':'timestamp'},inplace=True)

In [98]:
# Dict_flights['NOK600-1670198400-schedule-0700'].between_time('23:00', '07:00')
if Dict_flights['NOK600-1670198400-schedule-0700'].iloc[:1].between_time('23:00', '07:00').empty :
    print(1)
else:
    print(0)
# Dict_flights['NOK600-1670198400-schedule-0700']

0


In [99]:
test_dict = {
        'day':{},
        'night':{}
    }

In [141]:
test_dict['night']['1'] = Dict_flights['SNY20-1670370011-adhoc-0'].copy()

In [142]:
test_dict['night']['1'][test_dict['night']['1']['altitude'] <= 50]

,timestamp,fa_flight_id,altitude,groundspeed,heading,latitude,longitude
0,2022-12-06 23:49:06,SNY20-1670370011-adhoc-0,3.0000,81.0000,209.0000,13.905900,100.604990
1,2022-12-06 23:49:07,SNY20-1670370011-adhoc-0,3.0625,81.6250,208.9375,13.905548,100.604803
2,2022-12-06 23:49:08,SNY20-1670370011-adhoc-0,3.1250,82.2500,208.8750,13.905196,100.604616
3,2022-12-06 23:49:09,SNY20-1670370011-adhoc-0,3.1875,82.8750,208.8125,13.904844,100.604429
4,2022-12-06 23:49:10,SNY20-1670370011-adhoc-0,3.2500,83.5000,208.7500,13.904492,100.604242
...,...,...,...,...,...,...,...
3963,2022-12-07 00:55:09,SNY20-1670370011-adhoc-0,3.5000,90.2500,208.2500,13.932567,100.620030
3964,2022-12-07 00:55:10,SNY20-1670370011-adhoc-0,3.3750,90.1875,208.1875,13.932196,100.619820
3965,2022-12-07 00:55:11,SNY20-1670370011-adhoc-0,3.2500,90.1250,208.1250,13.931824,100.619610
3966,2022-12-07 00:55:12,SNY20-1670370011-adhoc-0,3.1250,90.0625,208.0625,13.931452,100.619400


In [139]:
test_dict['night']['1']

,timestamp,fa_flight_id,altitude,groundspeed,heading,latitude,longitude
0,2022-12-06 23:49:06,SNY20-1670370011-adhoc-0,3.0000,81.0000,209.0000,13.905900,100.604990
1,2022-12-06 23:49:07,SNY20-1670370011-adhoc-0,3.0625,81.6250,208.9375,13.905548,100.604803
2,2022-12-06 23:49:08,SNY20-1670370011-adhoc-0,3.1250,82.2500,208.8750,13.905196,100.604616
3,2022-12-06 23:49:09,SNY20-1670370011-adhoc-0,3.1875,82.8750,208.8125,13.904844,100.604429
4,2022-12-06 23:49:10,SNY20-1670370011-adhoc-0,3.2500,83.5000,208.7500,13.904492,100.604242
...,...,...,...,...,...,...,...
3963,2022-12-07 00:55:09,SNY20-1670370011-adhoc-0,3.5000,90.2500,208.2500,13.932567,100.620030
3964,2022-12-07 00:55:10,SNY20-1670370011-adhoc-0,3.3750,90.1875,208.1875,13.932196,100.619820
3965,2022-12-07 00:55:11,SNY20-1670370011-adhoc-0,3.2500,90.1250,208.1250,13.931824,100.619610
3966,2022-12-07 00:55:12,SNY20-1670370011-adhoc-0,3.1250,90.0625,208.0625,13.931452,100.619400


In [102]:
for item in Dict_flights:
    print(item)

NOK600-1670198400-schedule-0700
NOK340-1670198400-schedule-0684
SNY20-1670370011-adhoc-0
NOK122-1670197440-schedule-0484
AIQ3188-1670197440-schedule-0483
AIQ311-1670197440-schedule-0491
AIQ3029-1670197440-schedule-0464
AIQ3102-1670197440-schedule-0478
TLM702-1670196540-schedule-0499
NOK550-1670195580-schedule-0452
TLM600-1670195580-schedule-0482
TLM506-1670195580-schedule-0485


In [124]:
Dict_flights[item]

KeyError: 'AIQ311-1670197440-schedule-0491'

In [123]:
for time in test_dict:
    print(time)
    for flight in test_dict[time]:
        print(flight)

day
night
afk
1


In [115]:
time

'night'

In [145]:
Dict_flights['NOK340-1670198400-schedule-0684'][Dict_flights['NOK340-1670198400-schedule-0684']['altitude'] <= 80]

,fa_flight_id,altitude,groundspeed,heading,latitude,longitude
2022-12-06 23:51:14,NOK340-1670198400-schedule-0684,3.000000,158.000000,209.000000,13.907960,100.602090
2022-12-06 23:51:15,NOK340-1670198400-schedule-0684,3.312500,158.125000,209.062500,13.907325,100.601720
2022-12-06 23:51:16,NOK340-1670198400-schedule-0684,3.625000,158.250000,209.125000,13.906690,100.601350
2022-12-06 23:51:17,NOK340-1670198400-schedule-0684,3.937500,158.375000,209.187500,13.906055,100.600980
2022-12-06 23:51:18,NOK340-1670198400-schedule-0684,4.250000,158.500000,209.250000,13.905420,100.600610
...,...,...,...,...,...,...
2022-12-07 00:32:24,NOK340-1670198400-schedule-0684,7.533333,130.866667,34.133333,15.213965,103.241958
2022-12-07 00:32:25,NOK340-1670198400-schedule-0684,7.400000,130.900000,34.100000,15.214456,103.242316
2022-12-07 00:32:26,NOK340-1670198400-schedule-0684,7.266667,130.933333,34.066667,15.214947,103.242674
2022-12-07 00:32:27,NOK340-1670198400-schedule-0684,7.133333,130.966667,34.033333,15.215439,103.243032


In [ ]:
# SNY20-1670370011-adhoc-0